## Example for sharpening UAV thermal imagery.

#### Note: 
This notebook showcase how to use pyDMS using UAV collected thermal and multispectral imagery. Thermal imagery has a coarser resolution than multispectral one. 

NOTE: UAV integrated multispectral and thermal sensors (like Altum and Altum PT) resample data using internal cubic convolution to match the thermal data to multispectral resolution (this has not been tested yet in this code).

#### First install these libraries in the environment (if running the first time)

We will need three images: multispectral, thermal and a mask (in this particular case, filled with the 255 value). The mask is to ensure multispectral and thermal images share a common area. Finally, we need a name for the resulting sharpenned thermal imagery.

Note that the multispectral imagery can also be a DSM, a vegetation index. The idea here is to have as much information about the surface at high resolution that a merged multispectral+DSM imagery is a good idea (not implemented here)

In [1]:
highResFilename = r"inputs\SLM_RGBNIR_vineyards_small.tif" # it can be multispectral, RGB, DSM, vegetation index.
lowResFilename = r"inputs\SLM_TIR_vineyards_small.tif"
lowResMaskFilename = r"inputs\SLM_TIR_mask_small.tif" # this can be original TIR * 0 + 1
outputFilename = r"outputs\TIR_sharpened200_True.tif"

### The configuration of pyDMS is relativey easy:

- algorithm to use: RandomForest (useDecisionTree = True) or Neural Networks (useDecisionTree = False)
- sharpening temperature: yes (disaggregatingTemperature = True) or another data (disaggregatingTemperature = False)
    

In [5]:
useDecisionTree = True # if False, a neural network sharpener will be used
disaggregatingTemperature = True # if false, it will not apply temperature radiometric transformation.
mask_values = 255 # pixels with this value in the lowResMaskFilename will be processed.

### Running the code...

In [6]:
import os
import time

from osgeo import gdal

import pyDMS.pyDMSUtils as utils
from pyDMS.pyDMS import DecisionTreeSharpener, NeuralNetworkSharpener
from pyDMS.pyDMS import REG_sknn_ann, REG_sklearn_ann


In [7]:
commonOpts = {"highResFiles":               [highResFilename],
                  "lowResFiles":                [lowResFilename],
                  "lowResQualityFiles":         [lowResMaskFilename],
                  "lowResGoodQualityFlags":     [mask_values], #this is the value of the mask that indicates good quality pixels
                  "cvHomogeneityThreshold":     0,
                  "movingWindowSize":           200,
                  "disaggregatingTemperature":  [disaggregatingTemperature]}

dtOpts =     {"perLeafLinearRegression":    True,
                  "linearRegressionExtrapolationRatio": 0.25}

sknnOpts =   {'hidden_layer_sizes':         (10,),
                  'activation':                 'tanh'}

nnOpts =     {"regressionType":             REG_sklearn_ann,
                  "regressorOpt":               sknnOpts}

start_time = time.time()

if useDecisionTree:
        opts = commonOpts.copy()
        opts.update(dtOpts)
        disaggregator = DecisionTreeSharpener(**opts)
else:
        opts = commonOpts.copy()
        opts.update(nnOpts)
        disaggregator = NeuralNetworkSharpener(**opts)

print("Training regressor...")
disaggregator.trainSharpener()
print("Sharpening...")
downscaledFile = disaggregator.applySharpener(highResFilename, lowResFilename)
print("Residual analysis...")
residualImage, correctedImage = disaggregator.residualAnalysis(downscaledFile, lowResFilename,
                                                                   lowResMaskFilename,
                                                                   doCorrection=True)
print("Saving output...")
highResFile = gdal.Open(highResFilename)
if correctedImage is not None:
        outImage = correctedImage
else:
        outImage = downscaledFile
    # outData = utils.binomialSmoother(outData)
outFile = utils.saveImg(outImage.GetRasterBand(1).ReadAsArray(),
                            outImage.GetGeoTransform(),
                            outImage.GetProjection(),
                            outputFilename)
residualFile = utils.saveImg(residualImage.GetRasterBand(1).ReadAsArray(),
                                 residualImage.GetGeoTransform(),
                                 residualImage.GetProjection(),
                                 os.path.splitext(outputFilename)[0] + "_residual" +
                                 os.path.splitext(outputFilename)[1])

outFile = None
residualFile = None
downscaledFile = None
highResFile = None

print(time.time() - start_time, "seconds")

Training regressor...
Homogeneity CV threshold: 0.19
Number of training elements for is 62500 representing 100% of avaiable low-resolution data.
Homogeneity CV threshold: 0.20
Number of training elements for is 75000 representing 100% of avaiable low-resolution data.
Homogeneity CV threshold: 0.19
Number of training elements for is 75000 representing 100% of avaiable low-resolution data.
Homogeneity CV threshold: 0.22
Number of training elements for is 75000 representing 100% of avaiable low-resolution data.
Homogeneity CV threshold: 0.25
Number of training elements for is 32000 representing 100% of avaiable low-resolution data.
Homogeneity CV threshold: 0.20
Number of training elements for is 75000 representing 100% of avaiable low-resolution data.
Homogeneity CV threshold: 0.21
Number of training elements for is 90000 representing 100% of avaiable low-resolution data.
Homogeneity CV threshold: 0.22
Number of training elements for is 90000 representing 100% of avaiable low-resolution 